## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle
import os
import json
from preprocessing import *
from search_by_inverted_index import *
from w2v_d2v_functions import *

In [2]:
with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

Всего в корпусе 1384 пары вопрос-ответ

In [3]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0]

['\nДобрый день.Мой сын гражданин Украины (ДНР),имеет вид на жительство в Р.Ф., кот.получил проживая с 2014 г. в Нижегородской области.В 2017г. переехал на постоянное место жительство в г.Ростов.Официально трудоустроился на одно из промышл.предприятий г.Ростова.Оформил временную регистрацию в Ростове.В УФМС предупредили,что по истечении 90 дней он должен либо постоянно прописаться либо покинуть территорию России.Прошу проконсультировать как быть дальше.(Вернуться домой в Донецк,но здесь идет война,работы нет.В Ростове он работает по специальности.Он инженер машиностроитель.)Временная прописка до 15 марта.  Если он сможет приобрести какую либо недвижимость,как долго будет решаться вопрос о его постоянной прописке в Ростове.Как в этом случае будет решаться вопрос с видом на жительство в Ростове? Не получится ли ,что приобретя квартиру,он не успеет в ней прописаться до окончании срока временной регистрации. С уважением Людмила Евгеньевна.\n',
 'Добрый вечер!Из Вашего вопроса вообще ничего

In [5]:
questions = [elem[0] for elem in qa_corpus]

with open('questions.json', 'w', encoding='utf-8') as fw:
    json.dump(questions, fw)

In [6]:
# with open('questions.json', 'r', encoding='utf-8') as f:
#     questions = json.load(f)

In [7]:
answers = [elem[1] for elem in qa_corpus]

with open('answers.json', 'w', encoding='utf-8') as fw:
    json.dump(answers, fw)

In [8]:
# with open('answers.json', 'r', encoding='utf-8') as f:
#     answers = json.load(f)

In [9]:
# %%time

# inverted_index, document_length = get_inverted_index(mystem, answers)

In [10]:
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    inverted_index = json.load(f)

In [11]:
with open('document_length.json', 'r', encoding='utf-8') as f:
    document_length = json.load(f)

In [12]:
path_to_dir = '/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/DH/dh/araneum_none_fasttextskipgram_300_5_2018/'
model_path = 'araneum_none_fasttextskipgram_300_5_2018.model'

w2v_model = FastText.load(path_to_dir + model_path)
w2v_model.init_sims(replace=True)

In [13]:
# %%time

# data_word2vec = save_w2v_base(answers, w2v_model, mystem)

In [14]:
with open('w2v_base.pkl', 'rb') as f:
    data_word2vec = pickle.load(f)

In [15]:
paragraphs = get_paragraphs(answers, mystem)

In [16]:
%%time

d2v_model = train_doc2vec(paragraphs, 1000)

2836
CPU times: user 14min 38s, sys: 2min 31s, total: 17min 10s
Wall time: 10min 58s


In [17]:
# with open('d2v_model.pkl', 'rb') as f:
#     d2v_model = pickle.load(f)

In [18]:
data_doc2vec = save_d2v_base(d2v_model, paragraphs)

In [19]:
with open('d2v_base.pkl', 'rb') as f:
    data_doc2vec = pickle.load(f)

In [20]:
def search(query, search_method, n_results=5, return_answer_text=False):
    
    query = preprocessing(query, del_stopwords=False)
    
    if search_method == 'inverted_index':
        search_result = get_search_result(query, inverted_index, answers, document_length, n_results)
    
    elif search_method == 'word2vec':
        search_result = search_w2v(query, w2v_model, data_word2vec, n_results)
    
    elif search_method == 'doc2vec':
        search_result = search_d2v(query, d2v_model, data_doc2vec, n_results)
    
    else:
        raise TypeError('unsupported search method')
    
    if not return_answer_text:
        return search_result
    
    results = [(index, answers[index]) for index in search_result]
    return results

In [24]:
def get_accuracy(questions, search_method, n_results=5):
    accuracy_score = 0
    answers_index = []
    
    for i, question in enumerate(tqdm_notebook(questions)):
    
        search_result = search(question, search_method)
        
        if i in search_result:
            accuracy_score += 1
            answers_index.append(i)
            
    final_accuracy = accuracy_score * 100 / len(questions)        
    return final_accuracy, answers_index

In [25]:
%%time

w2v_score, w2v_res = get_accuracy(questions, 'word2vec')

HBox(children=(IntProgress(value=0, max=1384), HTML(value='')))


CPU times: user 45.9 s, sys: 1.99 s, total: 47.9 s
Wall time: 58.9 s


In [26]:
w2v_score

21.170520231213874

In [27]:
%%time

d2v_score, d2v_res = get_accuracy(questions, 'doc2vec')

HBox(children=(IntProgress(value=0, max=1384), HTML(value='')))


CPU times: user 2min 2s, sys: 1.53 s, total: 2min 3s
Wall time: 2min 14s


In [28]:
d2v_score

5.780346820809249

In [ ]:
%%time

inv_index_score, inv_index_res = get_accuracy(questions, 'inverted_index')

In [ ]:
inv_index_score